# Reconocimiento de dígitos con KNN



Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [ ]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

In [ ]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum

## Explorando los datos

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd

df_train = pd.read_csv("../data/train.csv")

Es un csv, con columnas por cada pixel

In [ ]:
df_train.shape

### Reducir el dataset

Ya que el dataset es grande, me quedo con una porción de éste

**<font color=red>ESTO ES SOLO PARA HACER PRUEBAS. CUANDO REPORTEN LOS RESULTADOS, USAR DATASET COMPLETO</font>**

In [ ]:
# data_train = df_train
data_train = df_train[:10000]
# data_test = df_train[5000:7000]

En el primer caracter está el dígito a reconocer. Llamamos a esto `y_train`

In [ ]:
# Uso values para mandar todo a arrays de numpy
X = data_train[data_train.columns[1:]].values
y = data_train["label"].values.reshape(-1, 1)

X.shape, y.shape

Convierto el resto a matrices de 28 x 28

A ver cómo se ven...

In [ ]:
import matplotlib.pyplot as plt

num = 8

print("Supuestamente es un {}".format(int(y[num])))

img = X[num].reshape(28, 28)
plt.imshow(img, cmap="Greys")


### Balance de clases

¿Tenemos la misma cantidad de cada número?

In [ ]:
data_train["label"].value_counts(normalize=True)

No: tenemos más 1 que 5, pero tampoco está tan desbalanceado

### Conjunto de development

In [ ]:
#DATA SPLIT: elijo que porcentaje del total es para training y cual es para Testeo.
#limit = int(0.8 * X.shape[0]) 

#X_train, y_train = X[:limit], y[:limit]
#X_test, y_test = X[limit:], y[limit:]

#assert len(X_train) == len(y_train)
#assert len(X_test) == len(y_test)

#print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_test)} de testeo")
#print("Ahora tengo {0} instancias de entrenamiento y {0} de testeo".format(len(X_train)))


In [ ]:
def list_mean(list):
    return sum(list)/ len(list)

#  K-Fold Validation


In [ ]:
#dividimos los datos de entrenamiento en K subsets. Utilizamos K-1 de esos subsets para entrenar a la maquina y
# el que queda se usa como validacion

#Se hacen K distintos rounds de entrenamiento variando el subset que se utiliza como validacion. Luego se testea
# cada uno de los 'rounds' y se calcula el Accuracy promedio entre todos ellos.

#no confundir K de K-fold con k de knn!!

from sklearn.model_selection import KFold

K = 5
kfold = KFold(n_splits = K)   #defino el split en K fold
kfold.get_n_splits(X)

print(kfold)


### KNN SIN PCA 

In [ ]:
#Entreno a la maquina con K-Fold y guardo los resultados para cada iteracion

from sklearn.metrics import accuracy_score
from time import time
#lista de los accuracies de cada round de K-Fold
allAcc= []

#neighbors
#ACA se puede jugar con la cantidad de Neighbors (k) que utiliza el algoritmo knn
k = 10
i=1



for train_index, test_index in kfold.split(X):
    #print(f"Train: {train_index} Val: {val_index}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #inicializamos un nuevo clasificador cada iteracion
    knn = metnum.KNNClassifier(k)
    
    fitStart = time()
    #entreno a la maquina con los K-1 subsets asignados
    knn.fit(X_train, y_train)
    fitEnd = time()
    fitTime = fitEnd - fitStart
    
    #ahora hago la prediccion con los X_val que se usan para validar la iteracion de entrenamiento actual
    predStart = time()
    y_pred = knn.predict(X_test)
    predEnd = time()
    predTime = predEnd - predStart
    #comparo los resultados de los X_val con sos correspondientes y_val
    acc = accuracy_score(y_test, y_pred)
    allAcc.append(acc)
    
    
    print("Round {}:".format(i))
    print("Accuracy: {}".format(acc))
    print("Fit time: {}".format(fitTime))
    print("Prediction time: {}".format(predTime))
    
    print("\n")
    i+= 1

meanAcc = list_mean(allAcc)
print("The mean accuracy of {}-Fold Cross-Validation with {} neighbors is: {}".format(K,k,meanAcc))

### Testeo de KNN sin PCA

In [ ]:
#Una vez entrenado el knn, se puede testear con los datos que nunca vio para ver si logro desarrollar 
# una prediccion generalizada (que no haya overfit)

#Tiempo de ejecucion
#%%time
#from sklearn.metrics import accuracy_score

#utilizo los X_test que la maquina nunca vio para testear
#y_pred = knn.predict(X_test)

#Accuracy
#test_acc = accuracy_score(y_test, y_pred)
#print("Accuracy: {}".format(test_acc))

#AGREGAR OTRO PARAMETRO MAS DE ANALISIS


### KNN CON PCA

In [ ]:
#me falta hacer lo mismo de arriba pero llamando a PCA 

In [ ]:
#Entreno a la maquina con K-Fold y guardo los resultados para cada iteracion

from sklearn.metrics import accuracy_score

#lista de los accuracies de cada round de K-Fold
allAcc= []



#neighbors
#ACA se puede jugar con la cantidad de Neighbors (k) que utiliza el algoritmo knn
k = 10
alpha = 30
i=1



for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #el sistema que vamos a entrenar
    knn = metnum.KNNClassifier(k)
    pca = metnum.PCA(alpha)
    pca.fit(X_train)
    
    X_train_transformed = pca.transform(X_train)
    
    #entreno a la maquina con los K-1 subsets asignados
    knn.fit(X_train_transformed, y_train)
       
    X_test_transformed = pca.transform(X_test)

    y_pred = knn.predict(X_test_transformed)
    
    #comparo los resultados de los X_val con sos correspondientes y_val
    acc = accuracy_score(y_test, y_pred)
    allAcc.append(acc)
    
    print("Round {}:".format(i))
    print("Accuracy: {}".format(acc))
    print("\n")
    i+= 1

meanAcc = list_mean(allAcc)
print("The mean accuracy of {}-Fold Cross-Validation with PCA alpha={} with {} neighbors is: {}".format(K,alpha,k,meanAcc))

In [ ]:
#NO CORRER


#X_transformed = pca.transform(X_test)
#y_pred = knn.predict(X_transformed)

#test_acc = accuracy_score(y_test, y_pred)
#print("Accuracy KNN with PCA: {}".format(test_acc))

### Testeo SIN PCA

#### (todo esto se puede ignorar)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Acá pueden cambiar el clasificador nuestro por el de sklearn!


#ACA se puede jugar con la cantidad de Neighbors (k) que utiliza el algoritmo knn
clf = metnum.KNNClassifier(50)
#clf = KNeighborsClassifier(100)

clf.fit(X_train, y_train)


In [ ]:
#Tiempo de ejecucion
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)


#Accuracy
acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

#AGREGAR OTRO PARAMETRO MAS DE ANALISIS

